In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 기본
import pandas as pd
import numpy as np
from functools import reduce

import xgboost as xgb
from sklearn.metrics import f1_score, classification_report
import matplotlib.pyplot as plt

# 그래프 설정
plt.rcParams['font.family'] = 'Malgun Gothic'

# 경고 뜨지 않게 설정
import warnings
warnings.filterwarnings('ignore')

In [ ]:
file_path = "/content/drive/MyDrive/12조 파이널프로젝트/data/train/201812_add_segment1.parquet"
df = pd.read_parquet(file_path)

In [ ]:
df

,ID,기준년월,소지카드수_유효_신용,소지카드수_이용가능_신용,입회경과개월수_신용,이용금액_R3M_신용_가족,이용여부_3M_해외겸용_본인,이용여부_3M_해외겸용_신용_본인,2순위신용체크구분_인코딩,CA한도금액,...,_3순위교통업종_이용금액,_1순위납부업종_이용금액,RP건수_B0M,RP유형건수_B0M,이용개월수_오프라인_R6M,연속유실적개월수_기본_24M_카드,할인건수_R3M,잔액_일시불_B2M,Segment,Segment1
0,TRAIN_000000,201812,1,1,71,0,0,0,0,5795,...,0,1970,1,1,6,17,0,554,D,other
1,TRAIN_000001,201812,1,1,16,0,0,0,-1,4236,...,0,2260,0,0,6,17,0,2898,E,E
2,TRAIN_000002,201812,1,1,128,0,0,0,0,28985,...,0,0,0,0,6,8,0,4369,C,other
3,TRAIN_000003,201812,2,2,31,0,1,1,0,10156,...,0,1961,0,0,6,24,0,1018,D,other
4,TRAIN_000004,201812,1,1,6,0,1,0,0,53912,...,0,0,0,0,0,0,0,0,E,E
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399995,TRAIN_399995,201812,1,1,209,0,1,0,0,10167,...,0,0,0,0,0,0,0,0,E,E
399996,TRAIN_399996,201812,1,1,17,0,1,1,-1,31159,...,0,161,2,2,6,17,0,4362,D,other
399997,TRAIN_399997,201812,1,1,115,0,1,1,-1,19429,...,0,0,0,0,6,24,0,2898,C,other
399998,TRAIN_399998,201812,1,1,71,0,0,0,-1,4228,...,0,0,0,0,0,0,0,0,E,E


In [ ]:
df= df.drop(columns=['Segment1'])

In [ ]:
df_filtered = df[df['Segment'].isin(['A', 'B'])].copy()  # A, B만 남긴 행 필터링
df['Segment3'] = df['Segment']  # Segment 값 복사해서 Segment3 만들기

In [ ]:
import glob
pd.set_option('display.max_columns', 105)  # 최대 105열까지 보여줌
#pd.reset_option('display.max_columns')

df_filtered

,ID,기준년월,소지카드수_유효_신용,소지카드수_이용가능_신용,입회경과개월수_신용,이용금액_R3M_신용_가족,이용여부_3M_해외겸용_본인,이용여부_3M_해외겸용_신용_본인,2순위신용체크구분_인코딩,CA한도금액,상향가능CA한도금액_1여부,최종이용일자_CA,최종이용일자_할부,최대이용금액_일시불_R12M,이용금액_해외,쇼핑_백화점_이용금액,쇼핑_마트_이용금액,쇼핑_편의점_이용금액,쇼핑_기타_이용금액,교통_정비이용금액,교통_택시이용금액,여유_운동이용금액,납부_기타이용금액,_1순위업종_이용금액,_2순위업종_이용금액,_3순위업종_이용금액,_2순위교통업종_이용금액,_3순위교통업종_이용금액,_1순위납부업종_이용금액,RP건수_B0M,RP유형건수_B0M,이용개월수_오프라인_R6M,연속유실적개월수_기본_24M_카드,할인건수_R3M,잔액_일시불_B2M,Segment
2898,TRAIN_002898,201812,1,1,136,0,1,1,-1,62925,0,20181222,20181226,19217,0,0,0,446,398,0,359,0,0,24334,8385,3855,359,0,0,2,1,6,24,0,12229,A
5253,TRAIN_005253,201812,2,2,305,0,1,1,1,54046,0,20181218,20181221,6788,0,0,1103,128,0,435,0,0,0,6283,2151,1822,435,0,0,0,0,6,24,0,2903,A
8128,TRAIN_008128,201812,3,3,95,0,1,1,1,52663,0,10101,20181217,45642,1618,0,0,679,0,643,310,0,523,78676,17955,6939,643,574,3001,4,2,6,24,0,19201,A
10808,TRAIN_010808,201812,3,3,218,9411,1,1,1,58361,0,20110306,20180924,78583,2532,1132,2141,500,0,463,280,0,332,33956,8662,7459,463,348,3564,3,2,6,24,0,21401,A
14951,TRAIN_014951,201812,1,1,65,0,1,1,-1,60071,0,20150704,20181207,66695,2097,1177,1332,160,0,316,210,1596,0,46370,17403,10045,602,316,0,2,2,6,24,0,16867,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
376373,TRAIN_376373,201812,1,1,259,0,1,1,-1,0,0,10101,20181224,75157,1722,1604,0,490,0,470,198,1924,319,77903,20182,10926,470,198,2445,2,1,6,24,0,11893,A
378479,TRAIN_378479,201812,3,3,186,12345,1,1,1,52255,0,10101,20180430,147539,1378,1563,3279,930,474,843,0,1666,299,90305,24961,17115,843,680,2114,1,1,6,24,1,24309,A
390620,TRAIN_390620,201812,1,1,221,0,1,1,0,19216,0,20181222,20181216,2981,0,0,0,0,0,0,0,0,0,3500,2448,903,0,0,2448,0,0,6,24,0,1450,B
393027,TRAIN_393027,201812,2,2,193,0,1,1,1,59353,0,20130608,20180421,143445,0,0,2286,340,0,705,211,3014,319,26122,18333,16418,705,211,319,1,2,6,24,0,47778,A


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
import xgboost as xgb

# 🔹 1. 데이터 불러오기
#file_path = "/content/drive/MyDrive/12조 파이널프로젝트/data/train/201812_add_segment1.parquet"
#df = pd.read_parquet(file_path)

# 🔹 2. Feature / Target 설정
X = df_filtered.drop(columns=['ID', '기준년월','Segment'], errors='ignore')
y = df_filtered['Segment']  # 'E' 또는 'other'

# 🔹 3. 라벨 인코딩 (문자 -> 숫자)
le = LabelEncoder()
y_encoded = le.fit_transform(y)  # 'E':0, 'other':1 (또는 반대)

# 🔹 4. 학습/검증 데이터 분리
X_train, X_val, y_train, y_val = train_test_split(
    X, y_encoded,
    test_size=0.2,
    stratify=y_encoded,
    random_state=42
)

# 🔹 5. XGBoost 모델 정의 및 학습
model = xgb.XGBClassifier(
    objective='binary:logistic',
    use_label_encoder=False,
    random_state=42
    #tree_method='hist'  # Colab에서 속도 빠르게
)

model.fit(X_train, y_train)

# 🔹 6. 예측 및 평가
y_pred = model.predict(X_val)
f1 = f1_score(y_val, y_pred, average='micro')
print(f"\n✅ [Validation F1 Score (micro)]: {f1:.4f}")
print("📘 클래스 인코딩 순서:", list(le.classes_))



✅ [Validation F1 Score (micro)]: 0.9211
📘 클래스 인코딩 순서: ['A', 'B']


In [ ]:
# 검증셋 예측
y_pred_val = model.predict(X_val).flatten()
y_pred_labels_val = le.inverse_transform(y_pred_val.astype(int))

# 검증 데이터 ID가 없다면, X_val 인덱스로 원본에서 ID 뽑기
submission_val = pd.DataFrame({
    'ID': df_filtered.loc[X_val.index, 'ID'],  # 검증셋 인덱스 기준으로 ID 가져오기
    'Segment': y_pred_labels_val
})

# 🔹 파일명 설정
file_name = "model3_xgb"  # 원하는 이름으로 변경 가능

# 🔹 csv로 저장
submission_path = f'/content/drive/MyDrive/12조 파이널프로젝트/이세윤/{file_name}_submission.parquet'
submission_val.to_parquet(submission_path, index=False)

print(f"✅ {file_name}_submission 저장 완료!")

✅ model3_xgb_submission 저장 완료!
